*Nombre Completo:* Victor Alberto Lizcano Portilla

*Cédula:* 1.094.270.723

*Correo Institucional:* alberto.lizcano@udea.edu.co


*Nombre Completo:* Jimmy Alexander Romero Miranda

*Cédula:* 1.017.203.451

*Correo Institucional:* jimmy.romero@udea.edu.co

#**Primera Iteración**

A continuación se muestra el proceso utilizado para generar el entrenamiento y la primera iteración de nuestro modelo de predicción de ventas.

Inicialmente se procede a clonar el repositorio disponible en *GitHub* https://github.com/Alberto7526/MonografiaGithub.git donde se encuentra alojado todos los archivos necesarios para la ejecución del proyecto.

In [1]:
!git clone https://github.com/Alberto7526/MonografiaGithub.git

Cloning into 'MonografiaGithub'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 46 (delta 12), reused 37 (delta 7), pack-reused 0
Unpacking objects: 100% (46/46), done.
Checking out files: 100% (13/13), done.


Importamos los paquetes necesarios para la ejecución del proyecto

In [14]:
import os
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib
import matplotlib.pyplot as plt
import seaborn as sns

##**Carga del dataset**

<p aling = justify>En el notebook anterior <i>01 - preprocesamiento.ipynb</i>, se realiza la verificación de los datos y limpieza de los mismos, eliminando datos outliers y valores nulos, por lo que se genera un nuevo archivo <i>sales_train.csv</i><p>. 
<p aling = justify>En esta sección cargamos el archivo con ayuda de pandas ubicado en la carpeta <i>New_Datasets</i>, debido al tamaño del archivo y la limitante en memoria presente en colab, solo se trabaja con las primeras 2.000.000 filas, como se indica en el código a continuación.<p>

In [3]:
sales_train = pd.read_csv('MonografiaGithub/New_Datasets/sales_train.csv')
sales_train['date'] = pd.to_datetime(sales_train['date'],errors='coerce', dayfirst=True)
sales_train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999.00,1.0
1,2013-01-03,0,25,2552,899.00,1.0
2,2013-01-05,0,25,2552,899.00,-1.0
3,2013-01-06,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935837,2015-10-10,33,25,7409,299.00,1.0
2935838,2015-10-09,33,25,7460,299.00,1.0
2935839,2015-10-14,33,25,7459,349.00,1.0
2935840,2015-10-22,33,25,7440,299.00,1.0


#**Agrupando las ventas por mes**

En esta sección se agrupa los ventas de cada producto por mes, tienda y producto, ya que en el modelo final se requere hacer la predicción de las ventas de x producto perteneciente a determinada tienda en un mes.  

In [4]:
sales_train = sales_train.groupby(by=['date_block_num','item_id'],as_index=False).agg(y=pd.NamedAgg(column='item_cnt_day', aggfunc='sum'))
sales_train

,date_block_num,item_id,y
0,0,19,1.0
1,0,27,7.0
2,0,28,8.0
3,0,29,4.0
4,0,32,299.0
...,...,...,...
233905,33,22162,10.0
233906,33,22163,26.0
233907,33,22164,15.0
233908,33,22166,11.0


#**Transformación del dataset de entrenamiento**

<p aling = justify>Para realizar las predicciones de las ventas futuras, se procede a organizar el dataset en filas, de tal manera que cada una de ellas lleve un historico en meses de las ventas y el mes actual como etiqueta para el dato a predecir.<p>



##**Función transformación**

La función transformación recibe como parametro el dataset y retorna un diccionario, donde cada llave corresponde al item del producto a predecir y los valores, son las ventas del producto organizadas por mes.  

In [5]:
def transformation(d):
    items = {}
    for i in d.item_id.unique():
      if not (i in items):
        items[i] = [0]*34
    for i in d.index:
      item = d.item_id[i]
      #shop = d.shop_id[i]
      block = d.date_block_num[i]
      items[item][block] = d.y[i]
    return (items)
    

##**Función timeseries**

Esta función tiene como parametros de entrada el *diccionario* generado en la función transformación y *step* que para esta caso será el número de meses historico que se tendrá en cuenta para realizar la predicción de las ventas en el mes siguiente.   

In [6]:
def timeseries(dictionary,step):
    data = {'item':[]}
    for i in range(step):
      data['y_'+str(i)]=[]
    data['y']=[]
    for clave,valor in dictionary.items():
      for k in range(0,len(valor)-step):
        data['item'].append(clave)
        for i in range(step):
          data['y_'+str(i)].append(int(valor[k+i]))
        data['y'].append(int(valor[k+step]))
    return data

Aplicamos las funciones mencionadas anteriormente y generamos un nuevo dataset

In [7]:
sales_train = pd.DataFrame(timeseries((transformation(sales_train)),4))

Debido a que el dataset no presenta información en ventas por mes de todos los productos contemplados inicialmente, se procede a filtrar aquellos que presenten una ventas de cero. 

In [8]:
train_filtered = sales_train[sales_train.y>0]
train_filtered

,item,y_0,y_1,y_2,y_3,y
30,27,7,3,4,1,2
31,27,3,4,1,2,2
32,27,4,1,2,2,3
33,27,1,2,2,3,1
34,27,2,2,3,1,1
...,...,...,...,...,...,...
654029,21973,0,0,0,0,11
654059,22004,0,0,0,0,44
654089,22005,0,0,0,0,30
654119,22006,0,0,0,0,30


#**División de los datos y entrenamiento del modelo**

Se divide el conjunto de datos en train y test, así como cada subconjunto en X y Y siendo X las caracteristicas y Y los datos deseado del modelo.  

In [9]:
train, test = train_test_split(train_filtered, test_size = 0.30)
Xtr, ytr = train[[i for i in train.columns if i!="y"]].values, train.y.values
Xts, yts = test[[i for i in test.columns if i!="y"]].values, test.y.values

In [10]:
estimator = RandomForestRegressor(n_estimators=120, max_depth=10,max_features=0.8)
estimator.fit(Xtr,ytr)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=10, max_features=0.8, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=120, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

Evaluación del modelo utilizando el coeficiente de determinación $R^2$

In [11]:
print ("train accuracy %.2f"%estimator.score(Xtr,ytr))
print ("test accuracy  %.2f"%estimator.score(Xts,yts))

train accuracy 0.80
test accuracy  0.63


Una vez se tiene el modelo entrenado se procede a guardarlo en la carpeta Model

In [17]:
try:
    os.mkdir('Model')
except OSError:
    pass
joblib.dump(estimator, 'Model/model.joblib')

['Model/model.joblib']

Para comprobar que el modelo guardado tiene un funcionamiento correcto, se procede a cargarlo y realizar una predicción con los datos de prueba.

In [22]:
model = joblib.load('Model/model.joblib')
predictions = modelo.predict(Xts)
print ("test accuracy with saved model  %.2f"%model.score(Xts,yts))

test accuracy with saved model  0.63


#**Métricas utilizadas**

- Para el proceso de entrenamiento se utilizó el error cuadrático medio como función de costo y para evaluar el modelo se utilizó el coeficiente de determinación $R^2$. 



###**Error cuadrático medio** 

Calcula el promedio de los errores al cuadrado, está definido por la siguiente ecuación: 

$ECM=\frac{1}{n}\sum_{i=1}^n(y_i-p_i)^2$

Donde $y_i$ corresponte al valor esperado, $p_i$ a la predicción hecha por el modelo y $n$ el número de muestras. 



###**Coeficiente de determinación $R^2$**

El método utilizado para evaluar el modelo es el coeficiente de determinación $R^2$, el cual permite medir que tan bien el modelo puede explicar los datos siendo 1 el valor óptimo y 0 el caso donde el modelo no tiene ninguna correlación con los datos. Esta definido por la siguiente ecuación. 

$R^2 = 1-\frac{u}{v}$

Donde: 

$u=\sum_{i=1}^n(y_i-p_i)^2$

$v=\sum_{i=1}^n(y_i-mean(p_i))^2$